# Uczenie ze wzmocnieniem
## Miłosz Mizak

### 1. Wstęp

Przedmiotem szóstego zadania była implementacja algorytmu Q-learning oraz zbadanie jego efektywności na podstawie środowiska **Taxi** z modułu **Gym**. Oprócz tego należało zbadać wpływ hiperparametru szybkości uczenia się na wyniki.

### 2. Implementacja

Algorytm został zaimplementowany w postaci klasy **Q_learning** w pliku **Qlearn.py**. Podczas inicjalizacji istnieje możliwość wczytania tabeli z wartościami nagród dla poszczególnych stanów (nazywaną dalej **tabelą Q**) z pliku json. Plik musi znajdować się w folderze wykonawczym oraz nosić nazwę **qtable.json**. 

Główną częścią implementacji jest funkcja *episode*. To w niej znajduje się właściwy algorytm Q-learning. Funkcja prezentuje się następująco:

In [ ]:
def episode(self, turns, pickMove):
    observation_old, info = self.env.reset()
    for _ in range(turns):
        move = pickMove(observation_old)
        observation_new, reward, terminated, truncated, info = self.env.step(move)
        self.table[observation_old][move] = self.table[observation_old][move] + self.learning_rate * (reward + self.discount_factor * max(self.table[observation_new]) - self.table[observation_old][move])
        observation_old = observation_new
        if terminated:
            break

Choć możliwym jest wywołanie funkcji *episode* samemu, zalecanym jest używanie funkcji *train*. Po przekazaniu jej ilości epizodów, maksymalnej ilości tur ruchu taksówki oraz wybraniu odpowiedniej strategii, algorytm przeprowadzi określoną ilość ćwiczeń.

In [ ]:
def train(self, episodes, turns, strategy):
    pickMove = None
    if strategy == "greedy":
        pickMove = self.greedyChoice
    elif strategy == "epsilonGreedy":
        pickMove = self.epsilonGreedyChoice
    elif strategy == "boltzmann":
        pickMove = self.boltzmannChoice
    elif strategy == "count":
        pickMove = self.countChoice
    else:
        raise ValueError("this strategy does not exist")
    for _ in range(episodes):
        self.episode(turns, pickMove)

Poszczególne strategie wyboru ruchu prezentują się następująco:
* strategia zachłanna - algorytm wybiera ruch o największej wartości nagrody,
* strategia E-zachłanna - algorytm z prawdopodobieństwem E wybiera losowy ruch, a z prawdopodobieństwem 1 - E: ruch zachłanny
* strategia oparta na rozkładzie Boltzmanna - każdy ruch otrzymuje określone prawdopodobieństwo na podstawie wartości nagród. Ruch jest następnie wybierany na podstawie tych prawdopodobieństw.
* strategia licznikowa - w 20% przypadków wybierany jest ruch, który do tej pory był wybierany najrzadziej. W przeciwnym wypadku używana jest strategia zachłanna.

W klasie Q_learning znajdują się również dwie metody, używane do testowania działania algorytmu. Są to odpowiednio *test* oraz *episodeTest*. Ta druga zachowuje się tak samo jak *episode*, z tą różnicą iż nie modyfikuje tabeli Q, pozwala więc dokładnie sprawdzić efektywność algorytmu. Zwraca także wynik epizodu - prawdę, jeśli taksówce udało się zawieźć pasażera do celu lub fałsz, jeżeli skończył się na to przeznaczony czas.

In [ ]:
def episodeTest(self, turns, pickMove):
    """
    Episode without changing the Qtable. Returns True if the goal has been reached
    and False if it hasn't been reached. It also returns the number of turns which it took
    to achieve the goal.
    """
    observation_old, info = self.env.reset()
    for i in range(turns):
        move = pickMove(observation_old)
        observation_new, reward, terminated, truncated, info = self.env.step(move)
        observation_old = observation_new
        if terminated:
            return (True, i+1)
    return (False, turns)

Metoda *test* jest nieco bardziej skomplikowana. Pozwala ona na przetestowanie każdej z dostępnych strategii. Dla każdej strategii generowane są wyniki, gdzie przedstawione jest ile epizodów zakończyło się porażką, ile sukcesem, oraz średnia ilość tur potrzebna do osiągnięcia tegoż sukcesu.

In [ ]:
from collections import namedtuple

def test(self, episodes, turns, useGreedy=False, useEpsilonGreedy=False, useBoltzmann=False, useCount=False):
    """
    Runs the algorithm without modyfing the Qtable. Depending on the function parameters,
    it is possible to use every strategy for testing the environment.
    Results may vary.
    """
    results = dict()
    Strategy = namedtuple('Strategy', ['use', 'f'])
    strategies = {
        "greedy" : Strategy(useGreedy, self.greedyChoice),
        "epsilonGreedy" : Strategy(useEpsilonGreedy, self.epsilonGreedyChoice),
        "boltzmann" : Strategy(useBoltzmann, self.boltzmannChoice),
        "count" : Strategy(useCount, self.countChoice)
    }
    for name, strategy in strategies.items():
        if strategy.use is True:
            miniResults = {"positive":0, "negative":0, "average":0}
            for _ in range(episodes):
                wasAchieved, numOfTurns = self.episodeTest(turns, strategy.f)
                if wasAchieved:
                    miniResults["positive"] += 1
                    miniResults["average"] += numOfTurns
                else:
                    miniResults["negative"] += 1
            if miniResults["positive"] != 0:
                miniResults["average"] = miniResults["average"] / miniResults["positive"]
            else:
                miniResults["average"] = 0
            results[f"{name}"] = miniResults
    return results

### 3. Testy

Aby zbadać działanie algorytmu oraz wpływ szybkości uczenia się na jego działanie, stworzyłem cztery funkcje testujące zawarte w pliku **Qtest.py**. Każda z nich testuje jedną z czterech strategii, a następnie generuje odpowiednie wykresy. Gdy strategia sama w sobie posiada pewien parametr który można modyfikować, dla każdego z tych wewnętrznych parametrów tworzony jest nowy wykres.

Wykresy pokazują, jak w czasie zmienia się efektywność algorytmu. Test za każdym razem składał się ze 100 epizodów, więc naturalnie efektywność była mierzona tym, ile z tych epizodów zakończyło się sukcesem.

## Wykresy

![greedy strategy](graphs/greedy1.png)

![epsilon greedy strategy 0.3](graphs/epsilon_greedy%2Cepsilon%3D0%2C3.png)

![epsilon greedy strategy 0.5](graphs/epsilon_greedy%2Cepsilon%3D0%2C5.png)

![epsilon greedy strategy 0.7](graphs/epsilon_greedy%2Cepsilon%3D0%2C7.png)

![boltzmann strategy 0.1](graphs/boltzmann%2Ctau%3D0%2C1.png)

![boltzmann strategy 1](graphs/boltzmann%2Ctau%3D1.png)

![boltzmann strategy 10](graphs/boltzmann%2Ctau%3D10.png)

![counting strategy](graphs/count.png)

# 4. Wnioski

Choć posczególne strategie różnią się efektywnością, to istnieje jeden wspólny mianownik. Gdy parametr uczenia się jest równy 1, algorytm uczy się najszybciej. Jest to zgodne z literaturą, gdzie taka wartość określana jest jako optymalna. Im wartość niższa od 1, tym algorytm wolniej się uczy. Co jednak ciekawe, zwiększenie tego parametru do 5 powoduje całkowite załamanie uczenia się. Najprawdopodobniej wynika to z tego, iż przy tak wysokim parametrze uczenia się stara wiedza jest całkowicie zamazywana przez nową. Nowe wartości powinny jedynie delikatnie zmieniać wartość nagrody, a nie stanowić jej większość.

Strategia zachłanna i E-zachłanna dają dobre wyniki, choć przy E-zachłannej należy uważać z parameterem E. Gdy jest równy 0.3, daje to najlepsze wyniki. Jest to dobry balans pomiędzy eksploracją przestrzeni a eksploatacją zdobytej już wiedzy. Strategia licznikowa także wykonuje swoje zadanie, choć zauważalnie gorzej niż strategie zachłanne.

Najciekawiej jednak działa strategia oparta na rozkładzie Boltzmanna. Niezależnie od parametru tau oraz od parametru uczenia się, wyniki są niezwykle chaotyczne. Nie jest to jednak zachowanie całkowicie losowe. Gdyby tak było, wykres powinien być linią prostą. Algorytm wykonuje czasami poprawne ruchy i czasami osiąga sukces, ale losowość jest najwyraźniej zbyt duża, aby był w stanie osiągać ten cel konsekwentnie.

Symptomy nadmiernej losowości można dostrzec na wykresie ze strategią E-zachłanną, z E równym 0.7. Choć algorytm uczy się, dochodząc do efektwności na poziomie 60%, to nie jest w stanie przekroczyć tej bariery. Eksploracja znacznie przeważa nad eksploatacją, więc algorytm nie jest w stanie dopracować tych dobrych ścieżek. Moim zdaniem strategia Boltzmannowska posiada ten sam problem, z tym że w dużo gorszym stopniu.